In [1]:
import pandas as pd
import os
import numpy as np
import logging
import sys
import torch
import copy
import yaml
import random

from prediction_utils.pytorch_utils.metrics import (
    StandardEvaluator,
    FairOVAEvaluator,
    CalibrationEvaluator
)

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style("ticks")

grp_label_dict = {1: 'Black women', 2: 'White women', 3: 'Black men', 4: 'White men'} 

args = {'cohort_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv',
        'base_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts',
        'grp_label_dict': {1: 'Black women', 2: 'White women', 3: 'Black men', 4: 'White men'},
        'plot_path': os.path.join('/labs/shahlab/projects/agataf/fairness_utility/eval/plots', 'paper_plots'),
        'eval_fold': 'test'
       }

calib_method = 'loess'


output_path = os.path.join(args['plot_path'], 'risk_category_counts.png')

def plot_calibration(calibration_df, zoom=False, output_path=None):  
    sns.set(font_scale=1.8)
    sns.set_style("ticks")
    df = calibration_df.assign(group = lambda x: x.group.map(grp_label_dict))
    g = sns.relplot(data = df,
                    x = 'pred_probs',
                    y = 'calibration_density',
                    kind = 'line',
                    hue = 'group',
                    col = 'model_type',
                    legend = True,
                    ci='sd',
                    err_style='bars',
                    aspect = 1)
    axes = g.axes.flatten()
    for ax in axes:
        ax.axline(xy1 = (0,0), slope = 1, color = "b", dashes = (5, 2), label = "Perfectly calibrated")
        ax.axvline(x = 0.075, linestyle = '--', color = 'grey')
        ax.axvline(x = 0.2,   linestyle = '--', color = 'grey')
        ax.axhline(y = 0.075, linestyle = '--', color = 'grey')
        ax.axhline(y = 0.2,   linestyle = '--', color = 'grey')

        if zoom:
            ax.set(xlim = (0, 0.3), ylim = (0, 0.3))
        else:
            ax.set(xlim = (0, 1), ylim = (0, 1))

        ax.set_xlabel("predicted risk")
        ax.set_ylabel("fraction of positives")
    
    if output_path is not None:
        g.savefig(output_path)

In [2]:
calibs_all = []
eqodds_threshold = 0.01
for experiment in ['original_pce', 'apr14_erm', 'apr14_erm_recalib', 'apr14_mmd']:

    aggregate_path = os.path.join(args['base_path'], 'experiments', 
                                      experiment, 'performance',
                                      'all', 'calibration', calib_method)
    #preds_path = os.path.join(aggregate_path, 'predictions.csv')
    #preds = pd.read_csv(preds_path)
    
    lin_calibs = pd.read_csv(os.path.join(aggregate_path, 'calibration_sensitivity_test_raw.csv'))
    if experiment in ['apr14_mmd', 'apr14_thr']:
        lin_calibs = lin_calibs.query('model_id == @eqodds_threshold')
        
    if 'model_id' not in preds.columns:
        lin_calibs = lin_calibs.assign(model_id=0)
    calibs_all.append(lin_calibs)   
    
calibs_all = pd.concat(calibs_all)

#     frac_treated = get_frac_treated(preds, args)  
#     frac_treated_all.append(frac_treated)
    

# include_recalibrated = False
# args = {'experiment_name': EXPERIMENT_NAME,
#         'cohort_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/cohort/all_cohorts.csv',
#         'base_path': '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts',
#         'n_bootstrap': 100,
#         'eval_fold': 'test'
#        }

# aggregate_path = os.path.join(args['base_path'], 'experiments', 
#                               EXPERIMENT_NAME, 'performance',
#                               'all')

# preds_path = os.path.join(aggregate_path, 'predictions.csv')
    
# preds = pd.read_csv(preds_path)
# if 'fold_id' not in preds.columns:
#     preds = preds.assign(fold_id=0)
# if 'model_id' not in preds.columns:
#     preds = preds.assign(model_id=0)

# lin_calibs = pd.read_csv(os.path.join(aggregate_path, 'calibration_sensitivity_test_raw.csv'))
# thr_calibs = pd.read_csv(os.path.join(aggregate_path, 'calibration_sensitivity_thresholds_raw.csv'))
    

NameError: name 'preds' is not defined

In [ ]:
plot_calibration(calibs_all, zoom=False)

In [ ]:
plot_calibration(calibs_all, zoom=True)

In [ ]:
    sns.set(font_scale=1.5)
    sns.set_style("ticks")
    zoom=False
    df = calibs_all.assign(group = lambda x: x.group.map(grp_label_dict))
    g = sns.relplot(data = df,
                    x = 'pred_probs',
                    y = 'calibration_density',
                    kind = 'line',
                    hue = 'group',
                    col = 'model_type',
                    legend = True,
                    ci='sd',
                    err_style='bars',
                    aspect = 1)
    axes = g.axes.flatten()
    for ax in axes:
        ax.axline(xy1 = (0,0), slope = 1, color = "b", dashes = (5, 2), label = "Perfectly calibrated")
        ax.axvline(x = 0.075, linestyle = '--', color = 'grey')
        ax.axvline(x = 0.2,   linestyle = '--', color = 'grey')
        ax.axhline(y = 0.075, linestyle = '--', color = 'grey')
        ax.axhline(y = 0.2,   linestyle = '--', color = 'grey')

        if zoom:
            ax.set(xlim = (0, 0.25), ylim = (0, 0.25))
        else:
            ax.set(xlim = (0, .8), ylim = (0, .8))

        ax.set_xlabel("predicted risk")
        ax.set_ylabel("fraction of positives")
    


In [ ]:
ax.errorbar

In [ ]:
calibs_all.query('model_type=="eqodds_mmd"')

In [ ]:
lin_calibs_short = lin_calibs.query("(lambda_reg==0) | (lambda_reg==0.001) | (lambda_reg==0.01) | (lambda_reg==0.1) | (lambda_reg==1)")



In [ ]:
lin_calibs

In [ ]:
    df = calibration_df.assign(group = lambda x: x.group.map(grp_label_dict))
    g = sns.relplot(data = df,
                    x = 'pred_probs',
                    y = 'calibration_density',
                    kind = 'line',
                    hue = 'group',
                    col = row_var,
                    legend = True,
                    ci='sd',
                    err_style='bars',
                    aspect = 1.2)
    axes = g.axes.flatten()
    for ax in axes:
        ax.axline(xy1 = (0,0), slope = 1, color = "b", dashes = (5, 2), label = "Perfectly calibrated")
        ax.axvline(x = 0.075, linestyle = '--', color = 'grey')
        ax.axvline(x = 0.2,   linestyle = '--', color = 'grey')
        ax.axhline(y = 0.075, linestyle = '--', color = 'grey')
        ax.axhline(y = 0.2,   linestyle = '--', color = 'grey')

        if zoom:
            ax.set(xlim = (0, 0.25), ylim = (0, 0.25))
        else:
            ax.set(xlim = (0, 1), ylim = (0, 1))

        ax.set_xlabel("predicted risk")
        ax.set_ylabel("fraction of positives")

In [ ]:
output_path = os.path.join(args['plot_path'], 'calibrations.png')
plot_calibration(lin_calibs_short, zoom=False, output_path=output_path, row_var=model_type_var)


output_path = os.path.join(args['plot_path'], 'calibrations_zoom.png')
plot_calibration(lin_calibs_short, zoom=True, output_path=output_path, row_var=model_type_var)



In [ ]:
a = thr_calibs.assign(group = lambda x: x.group.map(grp_label_dict))

if include_recalibrated:
    a = a.assign(calibrated = lambda x: x.calibrated.map({True: 'calibrated', False: 'uncalibrated'}))
else:
    a = a.assign(lambda_reg = lambda x: x.lambda_reg.astype(str))
    
output_path = os.path.join(args['plot_path'], 'real_thresholds.png')
    
    
g = sns.relplot(data = a,
                    x = model_type_var,
                    y = 'calibration_density',
                     col = 'pred_probs',
                   hue = 'group',
                    facet_kws = {'sharey': False, 'sharex': False},# 'margin_titles': True},
                    legend = True,
                    ci='sd',
                    kind="line",
                    err_style='bars',
                    aspect = 1.2)
axes = g.axes.flatten()
axes[0].axhline(y = 0.075, linestyle = '--', color = 'grey')

axes[1].axhline(y = 0.075, linestyle = '--', color = 'grey')
axes[1].axhline(y = 0.2,   linestyle = '--', color = 'grey')
        
if output_path is not None:
    g.savefig(output_path)